Initial Setup

In [2]:
import pandas as pd
import plotly.express as px
from pathlib import Path
from src.data import *
from src.metrics import *

FILE_PATH = Path("data/sample_datasets.xlsx")

data = load_xlsx(FILE_PATH)


Load and Clean Data

In [3]:
customers = data["customers"]
apps = data["applications"]
stores = data["stores"]
marketing = data["marketing"]

customers = customers.drop(customers.columns[0], axis=1)
apps = apps.drop(apps.columns[0], axis=1)
stores = stores.drop(stores.columns[0], axis=1)
marketing = marketing.drop(marketing.columns[0], axis=1)

apps["submit_date"] = pd.to_datetime(apps["submit_date"], errors="raise")

### Question 1: Application Volume

In [4]:
trend = app_trends_by_period(apps, freq="ME")

trend = trend.rename(columns={

    "submit_date": "Submission Date"
})
fig = px.line(
        trend,
        x="Submission Date",
        y=["applications","approved","used"],
        markers=True,
        title="Application Lifecycle Trends Over Time"
    )
fig.update_layout(
    yaxis_title="Application Count",
    xaxis_title="Submission Date",
    legend_title="Metric"
)

fig

In [5]:
totals = (
    trend[["applications", "approved", "used"]]
    .sum()
    .to_frame(name="Total")
    .reset_index()
    .rename(columns={"index": "Metric"})
)

totals.style.set_caption("Overall Application Totals")

,Metric,Total
0,applications,65535
1,approved,32756
2,used,16274


### Question 2: Approved vs Used

In [6]:
trend = app_trends_by_period(apps, freq="ME")
trend = trend.rename(columns={
    "avg_approved_amount": "Avg Approved Amount",
    "avg_used_amount": "Avg Used Amount",
    "submit_date": "Submission Date"
})

fig = px.line(
        trend,
        x="Submission Date",
        y=["Avg Approved Amount","Avg Used Amount"],
        markers=True,
        title="Average Amount Used vs Approved by Month",
        
    )
fig.update_layout(
    yaxis_title="Approved Dollars",
    xaxis_title="Submission Date",
    legend_title="Metric"
)

fig


### Question 3: Metrics by Store

In [7]:
appsCopy = apps.copy()
storesCopy = stores.copy()

combinedTable = appsCopy.merge(
    storesCopy,
    on="store",
    how="inner"
)

metricsTable = app_metrics_by_store(combinedTable)
metricsTable = metricsTable.sort_values(
    by="applications",
    ascending=False
)
metricsTable = (
    metricsTable
        .head(10)
        .style
        .set_caption("Application Metrics by Store (Top 10)")
        .format({
            "avg_pct_used": "{:.2%}",
            "avg_approved_amount": "${:,.2f}",
            "avg_used_amount": "${:,.2f}"
        })

)
metricsTable


,store,applications,approved,used,avg_approved_amount,avg_used_amount,avg_pct_used
165,store_247,327,175,101,"$3,104.01","$1,558.49",56.25%
131,store_216,304,160,70,"$3,052.74","$1,539.64",61.71%
197,store_51,304,160,89,"$3,047.14","$1,596.15",57.60%
191,store_46,302,133,75,"$3,126.35","$1,663.75",63.21%
242,store_92,302,157,80,"$3,073.99","$1,625.54",61.17%
144,store_228,300,145,69,"$2,957.74","$1,607.58",61.39%
112,store_2,300,157,73,"$3,057.14","$1,583.44",59.07%
73,store_164,300,139,72,"$3,106.06","$1,583.86",62.58%
177,store_33,299,149,79,"$3,013.32","$1,572.72",57.13%
34,store_129,297,147,63,"$3,308.73","$1,649.95",58.34%


### Question 4: Metrics by Marketing Campaign

In [8]:
appsCopy = apps.copy()
customersCopy = customers.copy()
marketingCopy = marketing.copy()

combinedTable = appsCopy.merge(
    customersCopy,
    on="customer_id",
    how="inner"
)

combinedTable = combinedTable.merge(
    marketingCopy,
    left_on="campaign",
    right_on="id",
    how="right"
)

metricsTable = app_metrics_by_Mktg(combinedTable)

metricsTable = (
    metricsTable
        .sort_values(by="marketing_spend", ascending=False)
        .rename(columns={
            "marketing_spend": "Marketing Spend",
            "used_amount": "Amount Used",
            "name": "Campaign"
        })
)

fig = px.bar(
    metricsTable,
    y="Campaign",  
    x=["Marketing Spend", "Amount Used"],  
    barmode="group",
    orientation="h",
    title="Marketing Spend vs Amount Used by Campaign"
)

fig.update_xaxes(tickprefix="$")
fig.update_yaxes(autorange="reversed")

fig.update_layout(
    xaxis_title="Dollar Amount",
    yaxis_title="Campaign",
    legend_title="Metric",
    template="plotly_white"
)

fig




### Question 5: Application Funnel

In [ ]:
submitted = len(apps)
approved = apps["approved_date"].notna().sum()
used = (apps["dollars_used"]> 0).sum()
approvalDrop = 1 - approved/submitted
usedDrop = 1 - used/approved

funnel_df = pd.DataFrame({
    "Stage" : ["Submitted", "Approved", "Used"],
    "Application Count" : [submitted, approved, used],
    "Dropoff": [
        f"",
        f"(▼ {approvalDrop:.1%})",
        f"(▼ {usedDrop:.1%})"
    ]
})


fig = px.funnel(
    funnel_df,
    x="Application Count",
    y="Stage",
    text="Dropoff",
    title="Application Conversion Funnel"
)

fig.update_traces(textposition="inside")
fig.update_layout(template="plotly_white")

fig

In [43]:
appsCopy = apps.copy()
storesCopy = stores.copy()

combinedTable = appsCopy.merge(
    storesCopy,
    on="store",
    how="inner"
)

combinedTable["submitted"] = 1
combinedTable["approved"] = combinedTable["approved_date"].notna().astype(int) # 1 if approved, 0 if approved_date is blank
combinedTable["used"] = (combinedTable["dollars_used"] > 0).astype(int)

metricsTable = (
    combinedTable
        .groupby("store", as_index=False)
        .agg(
            Submitted=("submitted", "sum"),
            Approved=("approved", "sum"),
            Used=("used", "sum")
        )
)

metricsTable["Approval Drop"] = 1 - metricsTable["Approved"] / metricsTable["Submitted"]
metricsTable["Usage Drop"] = 1 - metricsTable["Used"] / metricsTable["Approved"]
metricsTable["Total Drop"] = 1 - metricsTable["Used"] / metricsTable["Submitted"]

fig = (
    metricsTable
        .sort_values("Total Drop", ascending=False)
        .head(10)
        .style
        .set_caption("Application Conversion Funnel by Store (Top 10)")
        .format({
            "Submitted": "{:,}",
            "Approved": "{:,}",
            "Used": "{:,}",
            "Approval Drop": "▼ {:.1%}",
            "Usage Drop": "▼ {:.1%}",
            "Total Drop": "▼ {:.1%}"
        })
)

fig


,store,Submitted,Approved,Used,Approval Drop,Usage Drop,Total Drop
195,store_5,249,122,44,▼ 51.0%,▼ 63.9%,▼ 82.3%
213,store_66,263,131,50,▼ 50.2%,▼ 61.8%,▼ 81.0%
248,store_98,260,114,50,▼ 56.2%,▼ 56.1%,▼ 80.8%
249,store_99,263,134,51,▼ 49.0%,▼ 61.9%,▼ 80.6%
209,store_62,280,139,56,▼ 50.4%,▼ 59.7%,▼ 80.0%
169,store_26,242,106,49,▼ 56.2%,▼ 53.8%,▼ 79.8%
145,store_229,249,111,51,▼ 55.4%,▼ 54.1%,▼ 79.5%
200,store_54,267,117,55,▼ 56.2%,▼ 53.0%,▼ 79.4%
101,store_19,252,123,52,▼ 51.2%,▼ 57.7%,▼ 79.4%
25,store_120,266,131,55,▼ 50.8%,▼ 58.0%,▼ 79.3%
